In [93]:
from nipype import Node, Workflow, SelectFiles, Function, JoinNode, IdentityInterface, MapNode
from nipype.interfaces.io import DataSink

In [102]:
subj_id_list = ['%02d'%i for i in range(1, 3)]
subj_id_node = Node(IdentityInterface(fields=['subj_id']), name='subj_id_node')
subj_id_node.iterables = [('subj_id', subj_id_list)]

In [103]:
templates_anat = {'anat': '/home/neuro/Data/Visual_search_task/BIDS_data/sub-{subj_id}/ses-test/anat/sub-{subj_id}_ses-test_anat_sT1W.nii'}
selectanat = Node(SelectFiles(templates_anat, sort_filelist=True), name='selectanat')

In [104]:
session_id_list = ['1', '2', '3', '4']
templates_func = {'func': '/home/neuro/Result/Visual_search_task/preproc_working_dir/preproc_result/func/_subject_id_{subj_id}_task_number_{session_id}/detrend.nii.gz'}
selectfunc = Node(SelectFiles(templates_func, sort_filelist=True), name='selectfunc')
selectfunc.iterables = [('session_id', session_id_list)]

In [105]:
def store_func(anat, func_list):
    return anat, func_list
sort_file_node = JoinNode(Function(input_names=['anat', 'func_list'], output_names=['anat', 'func_list'], function=store_func), 
                          joinfield=['func_list'], joinsource=selectfunc, name='sort_file_node')

In [110]:
datasink = Node(DataSink(base_directory='/home/neuro/Result/Visual_search_task/test_workflow', 
                container='datasink'), name="datasink")

In [111]:
test_workflow = Workflow(base_dir='/home/neuro/Result/Visual_search_task/test_workflow', name='test_workflow')
test_workflow.connect([(subj_id_node, selectanat, [('subj_id', 'subj_id')]),
                       (subj_id_node, selectfunc, [('subj_id', 'subj_id')]), 
                       (selectanat, sort_file_node, [('anat', 'anat')]), 
                       (selectfunc, sort_file_node, [('func', 'func_list')]),
                       (sort_file_node, datasink, [('anat', 'anat'), 
                                                   ('func_list', 'func_list')])
                      ])

In [108]:
test_workflow.write_graph(graph2use='exec', simple_form=True)

211116-08:00:21,937 nipype.workflow INFO:
	 Generated workflow graph: /home/neuro/Result/Visual_search_task/test_workflow/test_workflow/graph.png (graph2use=exec, simple_form=True).


'/home/neuro/Result/Visual_search_task/test_workflow/test_workflow/graph.png'

In [112]:
test_workflow.run()

211116-08:05:18,383 nipype.workflow INFO:
	 Workflow test_workflow settings: ['check', 'execution', 'logging', 'monitoring']
211116-08:05:18,425 nipype.workflow INFO:
	 Running serially.
211116-08:05:18,426 nipype.workflow INFO:
	 [Node] Setting-up "test_workflow.selectfunc" in "/home/neuro/Result/Visual_search_task/test_workflow/test_workflow/_subj_id_02/_session_id_4/selectfunc".
211116-08:05:18,467 nipype.workflow INFO:
	 [Node] Running "selectfunc" ("nipype.interfaces.io.SelectFiles")
211116-08:05:18,489 nipype.workflow INFO:
	 [Node] Finished "test_workflow.selectfunc".
211116-08:05:18,490 nipype.workflow INFO:
	 [Node] Setting-up "test_workflow.selectfunc" in "/home/neuro/Result/Visual_search_task/test_workflow/test_workflow/_subj_id_02/_session_id_3/selectfunc".
211116-08:05:18,529 nipype.workflow INFO:
	 [Node] Running "selectfunc" ("nipype.interfaces.io.SelectFiles")
211116-08:05:18,551 nipype.workflow INFO:
	 [Node] Finished "test_workflow.selectfunc".
211116-08:05:18,552 nip

In [26]:
# String template with {}-based strings
templates = {'func': '/home/neuro/Result/Visual_search_task/preproc_working_dir/preproc_result/func/_subject_id_{subj_id}_task_number_{session_id}/detrend.nii.gz'}
selectfiles = Node(SelectFiles(templates, sort_filelist=True), name='selectfiles')

In [27]:
# list of subject identifiers
subject_list = ['%02d'%i for i in range(1, 3)]
session_list = ['1', '2', '3', '4']
selectfiles.iterables = [('subj_id', subject_list), ('session_id', session_list)]

In [28]:
def combine_image(anat_list, func_list):
    return anat_list[0], func_list

collecter = JoinNode(Function(input_names=['anat_list', 'func_list'], output_names=['anat', 'func_list'], function=combine_image),
             joinsource='selectfiles',
             joinfield=['anat_list', 'func_list'],
             name="collecter")

In [29]:
# Initiate the datasink node
output_folder = 'datasink'
datasink = Node(DataSink(base_directory='/home/neuro/Result/Visual_search_task/test_workflow', container=output_folder), name="datasink")

In [30]:
test_workflow = Workflow(base_dir='/home/neuro/Result/Visual_search_task/test_workflow', name='test_workflow')

In [31]:
test_workflow.connect([(selectfiles, collecter, [('anat', 'anat_list'),
                                                 ('func', 'func_list')]),
                       (collecter, datasink, [('anat', 'anat_file')])
                      ])

In [35]:
test_workflow.write_graph(graph2use='exec', simple_form=True)

211115-14:05:09,958 nipype.workflow INFO:
	 Generated workflow graph: /home/neuro/Result/Visual_search_task/test_workflow/test_workflow/graph.png (graph2use=exec, simple_form=True).


'/home/neuro/Result/Visual_search_task/test_workflow/test_workflow/graph.png'

In [33]:
test_workflow.run()

211115-13:23:28,329 nipype.workflow INFO:
	 Workflow test_workflow settings: ['check', 'execution', 'logging', 'monitoring']
211115-13:23:28,356 nipype.workflow INFO:
	 Running serially.
211115-13:23:28,357 nipype.workflow INFO:
	 [Node] Setting-up "test_workflow.selectfiles" in "/home/neuro/Result/Visual_search_task/test_workflow/test_workflow/_session_id_4_subj_id_02/selectfiles".
211115-13:23:28,393 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
211115-13:23:28,417 nipype.workflow INFO:
	 [Node] Finished "test_workflow.selectfiles".
211115-13:23:28,418 nipype.workflow INFO:
	 [Node] Setting-up "test_workflow.selectfiles" in "/home/neuro/Result/Visual_search_task/test_workflow/test_workflow/_session_id_3_subj_id_02/selectfiles".
211115-13:23:28,454 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
211115-13:23:28,478 nipype.workflow INFO:
	 [Node] Finished "test_workflow.selectfiles".
211115-13:23:28,4